In [ ]:
II
1. Introduction
1 Introduction
The aim of this tutorial is to provide detailed descriptions of funtion outputs
and features not covered (or covered only briefly) in the introductory
tutorial vignette("ST_intro", package="SpatioTemporal"). The reader
is encouraged to first study that tutorial.
As always this tutorial can be accesed from R with vignette("ST_tutorial",
package="SpatioTemporal") and the R-code can be found as edit( vignette("ST_tutorial",
package="SpatioTemporal") ) or Stangle( vignette("ST_tutorial",
package="SpatioTemporal")$file )
The remainder of this introduction provides some brief comments on common
problems that may arise when using the package (subsection 1.1) followed by
overviews of the data used in the examples of this tutorial (subsection 1.2) and
the theory behind the model (subsection 1.3). Following this background the
actual R-tutorial begins in section 2 with an overview of the data structures
used by the package to encapsulate data for the model fitting. Functions
that do parameter estimation and prediction are introduced in section 4,
along with tools for illustration of the results. The last part of the R-tutorial
is a cross-validation example in section 5.
The Appendices contain commented code for some additional examples: Appendix
A gives an example of predictions at unobserved locations and times,
Appendix B provides the outlines of a simulation study, and an MCMC example
is given in Appendix C.
1.1 Common Problems — Troubleshooting
Before starting with the full tutorial it seems prudent to discuss some of
the common problems that might arise when using the package, along with
possible solutions.
If the parameter estimation fails consider:
 Covariate scaling: avoid covariates with extremely different ranges; this
may cause numerical instabilities.
 The meaning of the parameters, compare the starting values to what
occurrs in the actual data.
 Try multiple starting points in the optimisation.
1
Tutorial for SpatioTemporal
 Changing location coordinates from kilometres to metres will drastically
change the reasonable values of the range.
 An over parameterised (too many covariates) model may cause numerical
problems.
Other common problems are:
 Ensure that geographic covariates are provided for all locations.
 Ensure that spatio-temporal covariates are provided for all time-points
and locations.
 The spatio-temporal covariate(s) must be in a list or 3D-array.

In [ ]:

1.2 Data
The data used in this tutorial consists of a subset of the NOx measurements
from coastal Los Angeles available to the MESA Air study; as well
as a few (geographic) covariates. A detailed description of the full dataset
can be found in Cohen et al. (2009), Szpiro et al. (2010), Lindstr¨om
et al. (2011, 2013) and a brief descriptions is given in vignette("ST_intro",
package="SpatioTemporal").
1.2.1 NOx Observations
The data consists of NOx-measurements from the national AQS network of
regulatory monitors as well as supplementary MESA Air (fixed site) monitoring.
The data has been aggregated to 2-week averages. Since the distribution
of the resulting 2-week average NOx concentrations (ppb) is skewed, the data
has also been log-transformed.
1.2.2 Geographic Covariates
To aid in the prediction at times and locations where we have no measurements
a set of spatial and/or spatio-temporal covariates can be used.
Covariates included in this example include both geographic covariates —
such as 1) distance to a major roads; 2) distance to coast (truncated to be
≤15km); and 3) average population density in a 2 km buffer — and a spatiotemporal
containing predictions from a deterministic air-pollution model —
Caline3QHC (EPA, 1992, Wilton et al., 2010, MESA Air Data Team, 2010).
2
1.3 Model and Theory
The covariates, and covariate selection, is described in detail by Mercer et al.
(2011), Cohen et al. (2009), and briefly in vignette("ST_intro", package="SpatioTemporal").
1.3 Model and Theory
The model and theory is described in vignette("ST_intro", package="SpatioTemporal")
or (Szpiro et al., 2010, Sampson et al., 2011, Lindstr¨om et al., 2011, 2013)
and the reader is referred to those papers for extensive details. We will inly
give a very brief overview here.
Denoting the quantity to be modelled (in this example ambient 2-week average
log NOx concentrations) by y(s, t), we write the spatio-temporal field
as
y(s, t) = µ(s, t) + ν(s, t), (1)
where µ(s, t) is the mean field and ν(s, t) is the essentially random space-time
residual field. The mean field is modelled as
µ(s, t) = X
L
l=1
γlMl(s, t) +Xm
i=1
βi(s)fi(t), (2)
where the Ml(s, t) are spatio-temporal covariates; γl are coefficients for the
spatio-temporal covariates; {fi(t)}
m
i=1 is a set of (smooth) temporal basis
functions, with f1(t) ≡ 1; and the βi(s) are spatially varying coefficients for
the temporal functions.
The βi(s)-coefficients in (2) are treated as spatial fields with a universal
kriging structure, allowing the temporal structure to vary between locations:
βi(s) ∈ N(Xiαi
, Σβi
(θi)) for i = 1, . . . , m, (3)
where Xi are n × pi design matrices, αi are pi × 1 matrices of regression
coefficients, and Σβi
(θi) are n × n covariance matrices. The Xi matrices often
contain geographical covariates and we dentote this component a “land use”
regression (LUR). This structure allows for different covariates and covariance
structures in the each of the βi(s) fields; the fields are assumed to be apriori
independent of each other.
The residual space-time field, ν(s, t), is assumed to be independent in time
3
Tutorial for SpatioTemporal
with stationary parametric spatial covariance
ν(s, t) ∈ N


0,



Σ
1
ν
(θν) 0 0
0
.
.
. 0
0 0 ΣT
ν
(θν)



| {z }
Σν(θν)


, (4)
Here the size of each block matrix, Σt
ν
(θν), is the number of observations, nt
,
at each time-point.
1.3.1 Model parameters
The parameters of the model consist of the regression parameters for the geographical
(α = (α
>
1
, . . . , α>
m)
>) and spatio-temporal covariates (γ = (γ1, . . . , γL)
>);
and covariance parameters for the βi- and ν-fields
θB = (θ1, . . . , θm) and θν.